In [1]:
test = '''
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input}

### Response:
'''

In [2]:
import os
import torch
# Set GPU device
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# os.environ['http_proxy']  = 'http://192.41.170.23:3128'
# os.environ['https_proxy'] = 'http://192.41.170.23:3128'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

c:\Users\Tairo Kageyama\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device(type='cpu')

In [3]:
# Step 1: Load the dataset
from datasets import load_dataset
from trl import SFTTrainer
import datasets
print(datasets.__version__)

# dataset = load_dataset("imdb", split="train")

# dataset

# file_path = r"C:\Users\Tairo Kageyama\Documents\GitHub\Python-fo-Natural-Language-Processing-main\lab8\alpaca_data.json"
file_path = r"C:\Users\Tairo Kageyama\Documents\GitHub\Python-fo-Natural-Language-Processing-main\lab8\alpaca_eval.json"
dataset = load_dataset("json", data_files=file_path, split="train")

2.18.0


In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM


model = AutoModelForCausalLM.from_pretrained("distilgpt2", device_map = 'auto')
tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
tokenizer.pad_token = tokenizer.eos_token

In [5]:
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['instruction'])):
        text = f"### Question: {example['instruction'][i]}\n ### Answer: {example['output'][i]}"
        output_texts.append(text)
    return output_texts


    # output_texts = []
    # for i in range(len(example['instruction'])):
    #     text =f"""
    #     Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

    #     ### Instruction:
    #     {example['instruction'][i]}

    #     ### Answer:
    #     {example['output'][i]}
    #     """.strip()
    # output_texts.append(text)
    # return output_texts
    

#check instruction-prompt
formatting_prompts_func(dataset[:2])

['### Question: What are the names of some famous actors that started their careers on Broadway?\n ### Answer: Some famous actors that started their careers on Broadway include: \n1. Hugh Jackman \n2. Meryl Streep \n3. Denzel Washington \n4. Julia Roberts \n5. Christopher Walken \n6. Anthony Rapp \n7. Audra McDonald \n8. Nathan Lane \n9. Sarah Jessica Parker \n10. Lin-Manuel Miranda',
 '### Question: How did US states get their names?\n ### Answer: US states get their names from a variety of sources, including Native American tribes, Spanish explorers, British colonists, and even presidents. For example, the state of Alabama was named after the Native American tribe that lived in the area, while the state of Florida gets its name from the Spanish explorer, Ponce de Leon, who explored the area in the 1500s. Other states are named after English kings (like Virginia, named after England\'s "Virgin Queen," Queen Elizabeth I) or presidents (like Washington, named after George Washington).']

In [6]:
response_template = " ### Answer:"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

trainer = SFTTrainer(
    model = model,
    train_dataset=dataset.select(range(300)),
    formatting_func=formatting_prompts_func,
    data_collator=collator,
)

trainer.train() 

c:\Users\Tairo Kageyama\AppData\Local\Programs\Python\Python310\lib\site-packages\trl\trainer\sft_trainer.py:245: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
100%|██████████| 114/114 [32:33<00:00, 17.14s/it]

{'train_runtime': 1953.516, 'train_samples_per_second': 0.461, 'train_steps_per_second': 0.058, 'train_loss': 2.21263965807463, 'epoch': 3.0}


TrainOutput(global_step=114, training_loss=2.21263965807463, metrics={'train_runtime': 1953.516, 'train_samples_per_second': 0.461, 'train_steps_per_second': 0.058, 'train_loss': 2.21263965807463, 'epoch': 3.0})

In [7]:
trainer.save_model(r"C:\Users\Tairo Kageyama\Documents\GitHub\Python-fo-Natural-Language-Processing-main\lab8\models\V2")

In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from datasets import load_dataset
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

model = AutoModelForCausalLM.from_pretrained("distilgpt2", device_map = 'auto')
tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
tokenizer.pad_token = tokenizer.eos_token

In [9]:
# alpaca_data


model2 = AutoModelForCausalLM.from_pretrained(r"C:\Users\Tairo Kageyama\Documents\GitHub\Python-fo-Natural-Language-Processing-main\lab8\models\V1",
                                              load_in_8bit = False)
model3 = AutoModelForCausalLM.from_pretrained(r"C:\Users\Tairo Kageyama\Documents\GitHub\Python-fo-Natural-Language-Processing-main\lab8\models\V2",
                                              load_in_8bit = False)
# prompt = "How many songs have been recorded throughout history? Try to explain your answer. Your explanation should take the reader through your reasoning step-by-step."
prompt = "How many songs have been recorded throughout history?"
input_ids = tokenizer.encode(prompt, return_tensors="pt")
max_length = 128
no_repeat_ngram_size = 3

outputs = model2.generate(input_ids=input_ids, max_length=max_length, no_repeat_ngram_size=no_repeat_ngram_size)
print(len(outputs[0]))
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
generated_text = generated_text.replace(prompt, "")
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


128



The song "I'm a Girl" is a song about a girl who is forced to accept the fact that she is a girl. She is forced into a relationship with a man who is a man and a woman. She has to accept that she has to be a man, and that she must be a woman to be able to be loved. She must be able and able to have a happy life. She needs to be happy, and she must have a good life.
The lyrics of the song "My Love Is My Love" are a perfect example of how the


In [10]:
# alpaca eval
input_ids = tokenizer.encode(prompt, return_tensors="pt")
max_length = 128
no_repeat_ngram_size = 3

outputs = model3.generate(input_ids=input_ids, max_length=max_length, no_repeat_ngram_size=no_repeat_ngram_size)
print(len(outputs[0]))
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
generated_text = generated_text.replace(prompt, "")
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


128



The first song, “The Last Time”, was recorded in 1887. It was recorded on a small island in the southern part of the island of Bering Sea. It is said to have been written by a man named John Bering, who was a member of the British Parliament. It has been recorded in the same way that the first song was recorded. It also has a number of other songs recorded throughout the history of the United Kingdom.

In 1887, the British government announced that it would be opening a new chapter in the British history of
